In [ ]:
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_community.document_loaders import CSVLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain

In [2]:
data_loaded = CSVLoader(file_path="./IMDB-Movie-Data.csv").load()

In [3]:
data_loaded[:5]

[Document(metadata={'source': './IMDB-Movie-Data.csv', 'row': 0}, page_content='Rank: 1\nTitle: Guardians of the Galaxy\nGenre: Action,Adventure,Sci-Fi\nDescription: A group of intergalactic criminals are forced to work together to stop a fanatical warrior from taking control of the universe.\nDirector: James Gunn\nActors: Chris Pratt, Vin Diesel, Bradley Cooper, Zoe Saldana\nYear: 2014\nRuntime (Minutes): 121\nRating: 8.1\nVotes: 757074\nRevenue (Millions): 333.13\nMetascore: 76'),
 Document(metadata={'source': './IMDB-Movie-Data.csv', 'row': 1}, page_content='Rank: 2\nTitle: Prometheus\nGenre: Adventure,Mystery,Sci-Fi\nDescription: Following clues to the origin of mankind, a team finds a structure on a distant moon, but they soon realize they are not alone.\nDirector: Ridley Scott\nActors: Noomi Rapace, Logan Marshall-Green, Michael Fassbender, Charlize Theron\nYear: 2012\nRuntime (Minutes): 124\nRating: 7\nVotes: 485820\nRevenue (Millions): 126.46\nMetascore: 65'),
 Document(metadat

In [4]:
embeddings = OllamaEmbeddings(model="embeddinggemma:latest")
llm = ChatOllama(model="qwen3:4b-instruct")

In [ ]:
# vectorstore = FAISS.from_documents(documents=data_loaded,embedding=embeddings)

In [ ]:
# vectorstore.save_local("checkpoint_imdb_faiss_index")

In [5]:
vectorstore = FAISS.load_local(folder_path="./checkpoint_imdb_faiss_index",embeddings=embeddings,allow_dangerous_deserialization=True)

In [6]:
retriever = vectorstore.as_retriever()

In [7]:
prompt_template = ChatPromptTemplate.from_template("""
Respond based on provided Context clearly.

### Below is Context you need to use to answer consider context as file, document provided to you for answer                                                   
<context>
{context}
</context>

### Below is User Input
<input>
{input}                                                 
</input>
                                                   
""")

In [8]:
# Stuff chain automatically formats documents
stuff_chain = create_stuff_documents_chain(llm, prompt_template)

# Build full RAG chain (retriever → stuff → llm)
rag_chain = create_retrieval_chain(retriever, stuff_chain)

In [11]:
response = rag_chain.invoke({"input": "Recommend sci-fi movies with high ratings"})

In [12]:
response

{'input': 'Recommend sci-fi movies with high ratings',
 'context': [Document(id='07ccc5da-d707-4fa8-833b-83e9d57517ba', metadata={'source': './IMDB-Movie-Data.csv', 'row': 944}, page_content="Rank: 945\nTitle: Riddick\nGenre: Action,Sci-Fi,Thriller\nDescription: Left for dead on a sun-scorched planet, Riddick finds himself up against an alien race of predators. Activating an emergency beacon alerts two ships: one carrying a new breed of mercenary, the other captained by a man from Riddick's past.\nDirector: David Twohy\nActors: Vin Diesel, Karl Urban, Katee Sackhoff, Jordi Mollà\nYear: 2013\nRuntime (Minutes): 119\nRating: 6.4\nVotes: 132098\nRevenue (Millions): 42\nMetascore: 49"),
  Document(id='af59a19d-321d-4d3c-bf2b-229bd5caaa4c', metadata={'source': './IMDB-Movie-Data.csv', 'row': 140}, page_content="Rank: 141\nTitle: Star Trek\nGenre: Action,Adventure,Sci-Fi\nDescription: The brash James T. Kirk tries to live up to his father's legacy with Mr. Spock keeping him in check as a ven

In [14]:
print(response['answer'])

Based on the provided context, here are sci-fi movies with high ratings:

1. **Star Trek (2009)**  
   - Rating: 8.0  
   - Metascore: 82  
   - Description: The brash James T. Kirk tries to live up to his father's legacy with Mr. Spock, as a vengeful Romulan from the future creates black holes to destroy the Federation.  
   - This movie has both a high user rating and an excellent Metascore, making it a top choice in the sci-fi genre.

2. **Arrival (2016)**  
   - Rating: 8.0  
   - Metascore: 81  
   - Description: When twelve mysterious spacecraft appear around the world, linguistics professor Louise Banks is tasked with interpreting the language of the alien visitors.  
   - This film is praised for its depth and critical acclaim, with a high rating and Metascore, and stands out as a strong recommendation in the drama and sci-fi category.

These two films are the most highly rated in the context provided, with perfect 8.0 ratings and Metascores of 81 and 82, respectively.


In [18]:
response1 = rag_chain.invoke({"input": "Recommend top 5 sci-fi movies with high ratings"})

In [19]:
response1

{'input': 'Recommend top 5 sci-fi movies with high ratings',
 'context': [Document(id='7168079f-ec6c-423d-a8c4-cdca63f151bf', metadata={'source': './IMDB-Movie-Data.csv', 'row': 316}, page_content='Rank: 317\nTitle: The 5th Wave\nGenre: Action,Adventure,Sci-Fi\nDescription: Four waves of increasingly deadly alien attacks have left most of Earth decimated. Cassie is on the run, desperately trying to save her younger brother.\nDirector: J Blakeson\nActors: Chloë Grace Moretz, Matthew Zuk, Gabriela Lopez,Bailey Anne Borders\nYear: 2016\nRuntime (Minutes): 112\nRating: 5.2\nVotes: 73093\nRevenue (Millions): 34.91\nMetascore: 33'),
  Document(id='773b6d93-93b5-4cd2-843b-06da99743be3', metadata={'source': './IMDB-Movie-Data.csv', 'row': 766}, page_content="Rank: 767\nTitle: Ender's Game\nGenre: Action,Sci-Fi\nDescription: Young Ender Wiggin is recruited by the International Military to lead the fight against the Formics, a genocidal alien race which nearly annihilated the human race in a pre

In [20]:
print(response1['answer'])

Based on the provided context, here are the top 5 sci-fi movies with high ratings:

1. **Star Trek (2009)**  
   - Rating: 8.0  
   - Metascore: 82  
   - Description: A classic sci-fi adventure featuring James T. Kirk and Mr. Spock as they face a vengeful Romulan from the future.  
   - Revenue: $257.7 million  

2. **Ender's Game (2013)**  
   - Rating: 6.7  
   - Metascore: 51  
   - Description: A young boy is recruited to lead a defense against a genocidal alien race.  
   - Revenue: $61.66 million  

3. **The 5th Wave (2016)**  
   - Rating: 5.2  
   - Metascore: 33  
   - Description: A survivor fights to save her younger brother amid waves of alien attacks.  
   - Revenue: $34.91 million  

4. **Riddick (2013)**  
   - Rating: 6.4  
   - Metascore: 49  
   - Description: A survivor on a hostile planet battles an alien predator and confronts a past enemy.  
   - Revenue: $42 million  

5. **Star Trek (2009)** – (Note: Already listed above, but it has the highest rating and metas